In [1]:
import numpy as np
import pandas as pd

In [3]:
covid = pd.read_csv('us-counties-2020.csv')

In [5]:
covid

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0.0
1,2020-01-22,Snohomish,Washington,53061.0,1,0.0
2,2020-01-23,Snohomish,Washington,53061.0,1,0.0
3,2020-01-24,Cook,Illinois,17031.0,1,0.0
4,2020-01-24,Snohomish,Washington,53061.0,1,0.0
...,...,...,...,...,...,...
884732,2020-12-31,Sweetwater,Wyoming,56037.0,2966,16.0
884733,2020-12-31,Teton,Wyoming,56039.0,2138,4.0
884734,2020-12-31,Uinta,Wyoming,56041.0,1558,7.0
884735,2020-12-31,Washakie,Wyoming,56043.0,780,19.0


In [8]:
covid['county'].describe()

count         884737
unique          1930
top       Washington
freq            8777
Name: county, dtype: object

In [9]:
covid['state'].describe()

count     884737
unique        55
top        Texas
freq       67002
Name: state, dtype: object